In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.init import xavier_normal_, xavier_uniform_
import argparse
import numpy as np
import time
import sys
from os.path import abspath

from base.graph_recommender import GraphRecommender
from util.sampler import next_batch_pairwise_kg, next_batch_pairwise
from util.conf import OptionConf
import torch
import torch.nn as nn 
import torch.nn.functional as F
from scipy.sparse import coo_matrix
from util.loss_torch import l2_reg_loss, EmbLoss, contrastLoss
from util.init import *
from base.torch_interface import TorchGraphInterface
import os
import numpy as np 
import time 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from itertools import product

from data.loader import FileIO
from util.conf import ModelConf
from base.recommender import Recommender
from data.ui_graph import Interaction
# from data.knowledge import Knowledge
from util.algorithm import find_k_largest
from time import strftime, localtime
from data.loader import FileIO
from util.evaluation import ranking_evaluation

import pandas as pd

/home/jun/anaconda3/envs/hungvv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
class GraphRecommender(Recommender):
    def __init__(self, conf, training_set, test_set, knowledge_set, **kwargs):
        super(GraphRecommender, self).__init__(conf, training_set, test_set, knowledge_set, **kwargs)
        self.data = Interaction(conf, training_set, test_set, knowledge_set, self.knowledge)
        self.bestPerformance = []
        top = self.ranking['-topN'].split(',')
        self.topN = [int(num) for num in top]
        self.max_N = max(self.topN)
        
        self.output = f"./results/LPHGTransformer/{self.dataset}/@{self.model_name}-inp_emb:{kwargs['input_dim']}-hyper_emb:{kwargs['hyper_dim']}-bs:{self.batch_size}-lr:{kwargs['lr']}-lrd:{kwargs['lr_decay']}-reg:{kwargs['reg']}-leaky:{kwargs['p']}-dropout:{kwargs['drop_rate']}-n_layers:{kwargs['n_layers']}-n_heads:{kwargs['n_heads']}-n_self_att:{kwargs['n_self_att']}/"
        if not os.path.exists(self.output):
            os.makedirs(self.output)

    def print_model_info(self):
        super(GraphRecommender, self).print_model_info()
        # # print dataset statistics
        print('Training Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.training_size()))
        print('Test Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.test_size()))
        print('=' * 80)

    def build(self):
        pass

    def train(self):
        pass

    def predict(self, u):
        pass

    def test(self, user_emb, item_emb):
        def process_bar(num, total):
            rate = float(num) / total
            ratenum = int(50 * rate)
            r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
            sys.stdout.write(r)
            sys.stdout.flush()

        # predict
        rec_list = {}
        user_count = len(self.data.test_set)
        for i, user in enumerate(self.data.test_set):
            # s_find_candidates = time.time()
            
            # candidates = predict(user)
            user_id  = self.data.get_user_id(user)
            score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
            candidates = score.cpu().numpy()
            
            # e_find_candidates = time.time()
            # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
            # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
            rated_list, li = self.data.user_rated(user)
            for item in rated_list:
                candidates[self.data.item[item]] = -10e8
            
            # s_find_k_largest = time.time()
            ids, scores = find_k_largest(self.max_N, candidates)
            # e_find_k_largest = time.time()
            # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
            item_names = [self.data.id2item[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
            if i % 1000 == 0:
                process_bar(i, user_count)
        process_bar(user_count, user_count)
        print('')
        return rec_list

    def evaluate(self, rec_list):
        self.recOutput.append('userId: recommendations in (itemId, ranking score) pairs, * means the item is hit.\n')
        for user in self.data.test_set:
            line = str(user) + ':'
            for item in rec_list[user]:
                line += ' (' + str(item[0]) + ',' + str(item[1]) + ')'
                if item[0] in self.data.test_set[user]:
                    line += '*'
            line += '\n'
            self.recOutput.append(line)
        current_time = strftime("%Y-%m-%d %H-%M-%S", localtime(time.time()))
        # output prediction result
        out_dir = self.output
        # out_dir = self.output['-dir']
        file_name = self.config['model.name'] + '@' + current_time + '-top-' + str(self.max_N) + 'items' + '.txt'
        FileIO.write_file(out_dir, file_name, self.recOutput)
        print('The result has been output to ', abspath(out_dir), '.')
        file_name = self.config['model.name'] + '@' + current_time + '-performance' + '.txt'
        self.result = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        self.model_log.add('###Evaluation Results###')
        self.model_log.add(self.result)
        FileIO.write_file(out_dir, file_name, self.result)
        print('The result of %s:\n%s' % (self.model_name, ''.join(self.result)))

    def fast_evaluation(self, model, epoch, user_emb, item_emb, kwargs=None):
        print('Evaluating the model...')
        s_test = time.time()
        rec_list = self.test(user_emb, item_emb)
        e_test = time.time() 
        print("Test time: %f s" % (e_test - s_test))
        
        s_measure = time.time()
        all_measures = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        len_measures = len(self.topN)
        lst_res = []
    
        data_ep = {}
        data_ep['epoch'] = epoch
        for i in range(0, len(all_measures), 5):
            mes = all_measures[i:i+5]
            topk = int(mes[0].split(' ')[1][:-1])
            hitk = float(mes[1].split(':')[1][:-1])
            preck = float(mes[2].split(':')[1][:-1])
            reck = float(mes[3].split(':')[1][:-1])
            ndcgk = float(mes[4].split(':')[1][:-1])
            data_ep[f'hit@{topk}'] = hitk
            data_ep[f'precision@{topk}'] = preck
            data_ep[f'recall@{topk}'] = reck
            data_ep[f'ndcg@{topk}'] = ndcgk

        measure = all_measures[(len_measures - 1) * 5: (len_measures - 1) * 5 + 5]
        e_measure = time.time()
        print("Measure time: %f s" % (e_measure - s_measure))
        if len(self.bestPerformance) > 0:
            count = 0
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            for k in self.bestPerformance[1]:
                if self.bestPerformance[1][k] > performance[k]:
                    count += 1
                else:
                    count -= 1
            if count < 0:
                self.bestPerformance[1] = performance
                self.bestPerformance[0] = epoch + 1
                self.save(model)
        else:
            self.bestPerformance.append(epoch + 1)
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            self.bestPerformance.append(performance)
            self.save(model)

        print('-' * 120)
        print('Real-Time Ranking Performance ' + ' (Top-' + str(self.max_N) + ' Item Recommendation)')
        measure = [m.strip() for m in measure[1:]]
        print('*Current Performance*')
        print('Epoch:', str(epoch + 1) + ',', '  |  '.join(measure))
        bp = ''
        # for k in self.bestPerformance[1]:
        #     bp+=k+':'+str(self.bestPerformance[1][k])+' | '
        bp += 'Hit Ratio' + ':' + str(self.bestPerformance[1]['Hit Ratio']) + '  |  '
        bp += 'Precision' + ':' + str(self.bestPerformance[1]['Precision']) + '  |  '
        bp += 'Recall' + ':' + str(self.bestPerformance[1]['Recall']) + '  |  '
        # bp += 'F1' + ':' + str(self.bestPerformance[1]['F1']) + ' | '
        bp += 'NDCG' + ':' + str(self.bestPerformance[1]['NDCG'])
        print('*Best Performance* ')
        print('Epoch:', str(self.bestPerformance[0]) + ',', bp)
        print('-' * 120)
        return measure, data_ep
    
    def save(self, model):
        with torch.no_grad():
            self.best_user_emb, self.best_item_emb = model()
        self.save_model(model)
    
    def save_model(self, model):
        # save model 
        current_time = strftime("%Y-%m-%d", localtime(time.time()))
        out_dir = self.output
        file_name =  self.config['model.name'] + '@' + current_time + '-weight' + '.pth'
        weight_file = out_dir + '/' + file_name 
        torch.save(model.state_dict(), weight_file)

    def save_loss(self, train_losses, rec_losses, reg_losses):
        df_train_loss = pd.DataFrame(train_losses, columns = ['ep', 'loss'])
        df_rec_loss = pd.DataFrame(rec_losses, columns = ['ep', 'loss'])
        df_reg_loss = pd.DataFrame(reg_losses, columns = ['ep', 'loss'])
        df_train_loss.to_csv(self.output + '/train_loss.csv')
        df_rec_loss.to_csv(self.output + '/rec_loss.csv')
        df_reg_loss.to_csv(self.output + '/reg_loss.csv')

    def save_perfomance_training(self, log_train):
        df_train_log = pd.DataFrame(log_train)
        df_train_log.to_csv(self.output + '/train_performance.csv')

## Model

In [5]:
class HGNNConv(nn.Module):
    def __init__(self, leaky, input_dim, hyper_dim):
        super(HGNNConv, self).__init__()
        self.hyper_dim = hyper_dim
        self.act = nn.LeakyReLU(negative_slope=leaky)
        self.fc1 = nn.Linear(input_dim, hyper_dim ,bias=False).to(device)
        self.fc2 = nn.Linear(hyper_dim, hyper_dim ,bias=False).to(device)
        self.fc3 = nn.Linear(hyper_dim, hyper_dim ,bias=False).to(device)

    def forward(self, adj, embeds):
        lat1 = self.act(adj.T @ embeds)
        lat2 = self.act(self.fc1(lat1)) +  lat1
        lat3 = self.act(self.fc2(lat2)) + lat2
        lat4 = self.act(self.fc3(lat3)) + lat3 
        ret = self.act(adj @ lat4)
        return ret

In [6]:
class DHCF(nn.Module):
    def __init__(self, config, data, args):        
        super(DHCF, self).__init__()
        
        self.data = data
        self.norm_adj = data.norm_adj
        self.sparse_norm_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(self.norm_adj).to(device)
        self._parse_args(args)
        
        hyper_uu = self.data.interaction_mat
        hyper_ii = self.data.interaction_mat.T
        
        self.hyper_uu = TorchGraphInterface.convert_sparse_mat_to_tensor(hyper_uu).to_dense().to(device)
        self.hyper_ii = TorchGraphInterface.convert_sparse_mat_to_tensor(hyper_ii).to_dense().to(device)
        # embeddings
        initializer = nn.init.xavier_uniform_
        self.user_embed = nn.Parameter(initializer(torch.Tensor(self.data.user_num, self.emb_size)).to(device))
        self.item_embed = nn.Parameter(initializer(torch.Tensor(self.data.item_num, self.emb_size)).to(device))
        # self.user_embed = nn.Parameter(torch.Tensor(self.data.user_num, self.emb_size)).to(device)
        # self.item_embed = nn.Parameter(torch.Tensor(self.data.item_num, self.emb_size)).to(device)
        
        # self.layers = [DHCFLayer(self.emb_size, self.emb_size, self.leaky, self.emb_size) for _ in range(self.n_layers)]
        self.dropout = nn.Dropout(self.drop_rate)
    
        self.hgnn_u = [HGNNConv(self.leaky, input_dim=self.hyper_dim, hyper_dim=self.hyper_dim) for _ in range(self.n_layers)]
        self.hgnn_i = [HGNNConv(self.leaky, input_dim=self.hyper_dim, hyper_dim=self.hyper_dim ) for _ in range(self.n_layers)]
        self.non_linear = nn.ReLU()
        
        self.fc_u = nn.Linear(self.emb_size, self.hyper_dim)
        self.fc_i = nn.Linear(self.emb_size, self.hyper_dim)
    
    def _parse_args(self, args):
        self.n_layers = args['n_layers']
        self.drop_rate = args['drop_rate'] 
        self.emb_size = args['input_dim']
        self.hyper_dim = args['hyper_dim']
        self.leaky = args['p']
        self.n_edges = args['hyper_num']
        
    def forward(self):
        user_emb = self.user_embed
        item_emb = self.item_embed
        
        user_emb = self.dropout(self.non_linear(self.fc_u(user_emb)))
        item_emb = self.dropout(self.non_linear(self.fc_i(item_emb)))
        
        user_out = user_emb.clone()
        item_out = item_emb.clone()
        
        for idx,layer in enumerate(range(self.n_layers)):
            user_hyper = torch.where(self.hyper_uu >= 0.5, 1., 0.)
            user_hyper = user_hyper.to(torch.float32)
            
            item_hyper = torch.where(self.hyper_ii >= 0.5, 1., 0.)
            item_hyper = item_hyper.to(torch.float32)
            
            hyperULat = self.hgnn_u[idx](user_hyper, user_emb)
            hyperILat = self.hgnn_i[idx](item_hyper, item_emb)
            
            # user_emb = self.dropout[idx](self.user_embed)
            # item_emb = self.dropout[idx](self.item_embed)
            #user_emb, item_emb = layer(self.H, user_emb, item_emb)
            # user_emb, item_emb = layer(self.hyper_uu, self.hyper_ii, user_emb, item_emb) 
            
            user_out = torch.cat((user_out, hyperULat), dim=1)
            item_out = torch.cat((item_out, hyperILat), dim=1)    
        return user_out, item_out

In [7]:
class DHCFLayer(nn.Module):
    def __init__(self, in_channels, out_channels, leaky, emb_size, bias=True):
        super(DHCFLayer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        # self.weight = nn.Parameter(torch.Tensor(in_channels, out_channels).to(device))
        self.hgnn_u = HGNNConv(leaky, input_dim=in_channels, hyper_dim=out_channels)
        self.hgnn_i = HGNNConv(leaky, input_dim=in_channels, hyper_dim=out_channels)
    
    def forward(self, H_u, H_i, user_emb, item_emb):
#         user_hyper = torch.cat((H, torch.matmul(H, torch.matmul(H.t(), H))), dim=1)
        user_hyper = H_u
        user_hyper = torch.where(user_hyper >= 0.5, 1., 0.)
        user_hyper = user_hyper.to(torch.float32)
        hyperULat = self.hgnn_u(user_hyper, user_emb)
        user_out = hyperULat 
        
        item_hyper = H_i
        item_hyper = torch.where(abs(item_hyper)>=0.5, 1., 0.)
        item_hyper = item_hyper.to(torch.float32)
        hyperILat = self.hgnn_i(item_hyper, item_emb)
        
        item_out = hyperILat
        
#         user_out = torch.matmul(M_u, self.weight) + self.bias
#         item_out = torch.matmul(M_i, self.weight) + self.bias
        
        return user_out, item_out

# Utility

## Loss

In [8]:
def calculate_loss(anchor_emb, pos_emb, neg_emb, batch_size, reg):
    def bpr_loss(user_emb, pos_item_emb, neg_item_emb):
        pos_score = torch.mul(user_emb, pos_item_emb).sum(dim=1)
        neg_score = torch.mul(user_emb, neg_item_emb).sum(dim=1)
        loss = -torch.log(10e-6 + torch.sigmoid(pos_score - neg_score))
        return torch.mean(loss)
    
    calc_reg_loss = EmbLoss()
    rec_loss = bpr_loss(anchor_emb, pos_emb, neg_emb)
    reg_loss = reg * calc_reg_loss(anchor_emb, pos_emb, neg_emb) / batch_size
    return rec_loss, reg_loss

In [9]:
def predict(u, rec, user_emb, item_emb):
    user_id  = rec.data.get_user_id(u)
    score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
    return score.cpu().numpy()  

## Train

In [10]:
def train():
    lst_train_losses = []
    lst_rec_losses = []
    lst_reg_losses = []
    lst_cl_losses = []
    lst_kg_losses = [] 
    lst_performances = []
    
    for ep in range(maxEpoch):
        train_losses = []
        rec_losses = []
        reg_losses = []
        cl_losses = []
        kg_losses = []
        
        for n, batch in enumerate(next_batch_pairwise(rec.data, batchSize)):
            user_idx, pos_idx, neg_idx = batch
            train_model.train()
            user_emb, item_emb = train_model()
            anchor_emb = user_emb[user_idx]
            pos_emb = item_emb[pos_idx]
            neg_emb = item_emb[neg_idx]
            
            rec_loss, reg_loss = calculate_loss(anchor_emb, pos_emb, neg_emb, batchSize, reg)
            # print("Rec loss", rec_loss)
            # print("Reg loss", reg_loss)
            batch_loss = rec_loss + reg_loss 
            
            train_losses.append(batch_loss.item())
            rec_losses.append(rec_loss.item())
            reg_losses.append(reg_loss.item())

            optimizer.zero_grad()
            batch_loss.backward()
            # torch.nn.utils.clip_grad_norm_(train_model.parameters(), 3)
            optimizer.step()

        batch_train_loss = np.mean(train_losses)
        scheduler.step(batch_train_loss)
        
        train_loss = np.mean(train_losses)
        rec_loss = np.mean(rec_losses)
        reg_loss = np.mean(reg_losses)
        
        lst_train_losses.append([ep, train_loss])
        lst_rec_losses.append([ep,rec_loss])
        lst_reg_losses.append([ep, reg_loss])

        # Evaluation
        train_model.eval()
        with torch.no_grad():
            user_emb, item_emb = train_model()
            _, data_ep = rec.fast_evaluation(train_model, ep, user_emb, item_emb)
        lst_performances.append(data_ep)

    rec.save_loss(lst_train_losses, lst_rec_losses, lst_reg_losses)
    rec.save_perfomance_training(lst_performances)
    user_emb, item_emb = rec.best_user_emb, rec.best_item_emb
    return user_emb, item_emb

## Test

In [11]:
def test(rec, user_emb, item_emb):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
        sys.stdout.write(r)
        sys.stdout.flush()

    # predict
    rec_list = {}
    user_count = len(rec.data.test_set)
    for i, user in enumerate(rec.data.test_set):
        # s_find_candidates = time.time()
        candidates = predict(user, rec, user_emb, item_emb)
        # e_find_candidates = time.time()
        # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
        # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
        rated_list, li = rec.data.user_rated(user)
        for item in rated_list:
            candidates[rec.data.item[item]] = -10e8

        # s_find_k_largest = time.time()
        ids, scores = find_k_largest(rec.max_N, candidates)
        # e_find_k_largest = time.time()
        # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
        item_names = [rec.data.id2item[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    rec.evaluate(rec_list)

## Main

In [12]:
model = 'DHCF'
config = ModelConf('./conf/' + model + '.conf')

dataset = 'ml-1m'
batchSize = int(config['batch_size'])
# lRate = float(config['learnRate'])
# maxEpoch = int(config['num.max.epoch'])
# reg = float(config['reg.lambda'])
maxEpoch = 500
lRates = [0.05]
lrDecays = [0.9]
regs = [0.1]
hyperDims = [128]
hyperNum = [128]
inputDims = [32]
ps = [0.1]
dropRates = [0.1]
nLayers = [2]
nHeads = [1]
nSelfAtt = [1]

if dataset == 'lastfm':
    training_set = "./dataset/lastfm/train.txt"
    test_set = './dataset/lastfm/test.txt'
    knowledge_data='./dataset/lastfm/processed/lastfm.kg'
elif dataset == 'ml-1m':
    training_set = './dataset/ml-1m/train.txt'
    test_set = './dataset/ml-1m/test.txt'
    knowledge_data = './dataset/ml-1m/processed/ml-1m.kg'
hyperparameters = [lRates, lrDecays, regs, hyperDims, inputDims, ps, dropRates, nLayers, hyperNum, nHeads, nSelfAtt]

# Run

In [ ]:
for params in product(*hyperparameters):
    lr, lr_decay, reg, hyper_dim, input_dim, prob, drop_rate, n_layers, hyper_num, n_heads, n_self_att = params
    print(params)
    args = {
        'lr': lr,
        'lr_decay': lr_decay,
        'reg': reg,
        'input_dim': input_dim,
        'hyper_dim': hyper_dim,
        'hyper_num': hyper_num,
        'p': prob,
        'drop_rate': drop_rate,
        'n_layers': n_layers,
        'n_heads': n_heads,
        'n_self_att': n_self_att,
        'dataset': dataset
    }
    training_data = FileIO.load_data_set(training_set, config['model.type'])
    test_data = FileIO.load_data_set(test_set, config['model.type'])
    knowledge_set = FileIO.load_kg_data(knowledge_data)

    rec = GraphRecommender(config, training_data, test_data, knowledge_set, **args)
    train_model = DHCF(rec.config, rec.data, args).to(device)
    optimizer  = torch.optim.Adam(train_model.parameters(), lr=lr,weight_decay=1e-3)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=lr_decay, patience=5)
    user_emb, item_emb = train()
    test(rec, user_emb, item_emb)
    

(0.05, 0.9, 0.1, 128, 32, 0.1, 0.1, 2, 128, 1, 1)
parameter ss_rate is not found in the configuration file!


/mnt/disk1/hungvv/recommender/SELFRec/base/torch_interface.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  i = torch.LongTensor([coo.row, coo.col])


Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 3.705601 s
Measure time: 0.081426 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 1, Hit Ratio:0.11546  |  Precision:0.05771  |  Recall:0.12877  |  NDCG:0.11299
*Best Performance* 
Epoch: 1, Hit Ratio:0.11546  |  Precision:0.05771  |  Recall:0.12877  |  NDCG:0.11299
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 1.031502 s
Measure time: 0.079197 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch

In [ ]:
# reg = 0.1
# args = {
#     'p':  0.1,
#     'drop_rate': 0.1,
#     'leaky': 0.1,
#     'n_layers': 2,
#     'input_dim': 64,
#     'hyper_dim': 128,
#     'hyperedge_num': 128,
#     'lr': 0.005,
#     'reg': reg,
#     'dataset': dataset
# }


# training_data = FileIO.load_data_set(training_set, config['model.type'])
# test_data = FileIO.load_data_set(test_set, config['model.type'])
# knowledge_set = FileIO.load_kg_data(knowledge_data)


# rec = GraphRecommender(config, training_data, test_data, knowledge_set, **args)
# if model == 'HGNN':
#     train_model = Model(rec.config, rec.data, args).to(device)
# elif model == 'LightGCN':
#     train_model = LGCN_Encoder(rec.data, args['input_dim'], args['n_layers'])
# elif model == 'HCCF':
#     train_model = HCCFEncoder(rec.data, args['input_dim'], args['n_layers'], args['hyperedge_num'], args['leaky'], args['drop_rate'])
# elif model == 'DHCF':
#     train_model = DHCFEncoder(rec.data, args['input_dim'], args['n_layers'], args['hyperedge_num'], args['drop_rate'], args['leaky'], args['hyper_dim'])
    
# optimizer  = torch.optim.Adam(train_model.parameters(), lr=args['lr'], weight_decay=0.001)
# scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=5)
# user_emb, item_emb = train(args)
# test(rec, user_emb, item_emb, args)

## Different hyperpamater settings

In [ ]:
# for idx, params in enumerate(product(*hyperparameters)):
#     lr, lr_decay, reg, hyper_dim, input_dim, prob, drop_rate, n_layers, hyperedge_num = params
#     print("hyperparameters:{}".format(params))
#     args = {
#         'p': prob,
#         'drop_rate': drop_rate,
#         'n_layers': n_layers,
#         'input_dim': input_dim,
#         'hyper_dim': hyper_dim,
#         'hyperedge_num': hyperedge_num,
#         'lr': lr,
#         'reg': reg,
#         'leaky': 0.5,
#         'dataset': dataset
#     }

#     training_data = FileIO.load_data_set(training_set, config['model.type'])
#     test_data = FileIO.load_data_set(test_set, config['model.type'])

#     rec = GraphRecommender(config, training_data, test_data, **args)
#     if model == 'HGNN':
#         train_model = Model(rec.config, rec.data, args).to(device)
#     elif model == 'LightGCN':
#         train_model = LGCN_Encoder(rec.data, input_dim, args['n_layers'])
#     elif model == 'HCCF':
#         train_model = HCCFEncoder(rec.data, input_dim, args['n_layers'], args['hyperedge_num'], args['leaky'], args['drop_rate'])
#     elif model == 'DHCF':
#         train_model = DHCFEncoder(rec.data, input_dim, args['n_layers'], args['hyperedge_num'], args['drop_rate'], args['leaky'], args['hyper_dim'])

#     optimizer  = torch.optim.Adam(train_model.parameters(), lr=lr)
#     scheduler = ReduceLROnPlateau(optimizer, 'min', factor=lr_decay, patience=5)
#     user_emb, item_emb = train(args)
#     test(rec, user_emb, item_emb, args)